<a href="https://colab.research.google.com/github/ShiroDima/TID-TIC045/blob/main/Distance_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Necessary Modules                                                

In [39]:
! pip install geopandas 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
import geopandas as gpd
from shapely.geometry import Point
import folium
import pandas as pd                                               

# Loading_Dataset

In [41]:
power_plants=gpd.read_file('/content/drive/MyDrive/TID_Innovation/Data/TID_Dataset/power_plants.csv',
                           GEOM_POSSIBLE_NAMES='geom',
                           KEEP_GEOM_COLUMNS='NO')

power_plants_2=gpd.read_file('/content/drive/MyDrive/TID_Innovation/Data/New folder/NGA_PowerPlants.shp')
                    
transmission_substations=gpd.read_file('/content/drive/MyDrive/TID_Innovation/Data/TID_Dataset/all_transmission_substations.csv',
                                        GEOM_POSSIBLE_NAMES='geom',
                                        KEEP_GEOM_COLUMNS='NO')

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.



In [42]:
# Converting the crs type of the geometry field so the distance is measured in meters                                   
power_plants_2=power_plants_2.set_crs('EPSG:4326')
power_plants_2=power_plants_2.to_crs('EPSG:3857')

In [43]:
transmission_substations

,FID,osm_id,power_substation,tags,geometry
0,all_transmission_substations.3114602,564813837,substation,{},POINT (966398.554 1049785.157)
1,all_transmission_substations.3114625,564466620,substation,{},POINT (971604.619 1051695.196)
2,all_transmission_substations.3117563,564468439,substation,{},POINT (987827.755 1060272.300)
3,all_transmission_substations.3128584,564469422,substation,{},POINT (990150.616 1067457.015)
4,all_transmission_substations.3130639,574213499,substation,{},POINT (986807.423 1094420.309)
...,...,...,...,...,...
163,all_transmission_substations.2798254,563728565,substation,{},POINT (759610.884 1115723.299)
164,all_transmission_substations.2798256,556550171,substation,"{""fixme"":""name,operator"",""substation"":""transmi...",POINT (760364.953 1115784.892)
165,all_transmission_substations.2945365,567187477,substation,{},POINT (820725.744 1171023.963)
166,all_transmission_substations.2967967,39469643,substation,"{""fixme"":""name,operator"",""substation"":""transmi...",POINT (825131.324 1185833.718)


# Calculator

In [44]:
def distance_calculator(longitude,latitude,n_closest):
  """This function calculates the distance between any given location and all power plants and transmission substation and returns the n_closest locations"""
  # Creating a shapely point geometry for the given location                                          
  location=Point(longitude,latitude)
  # The power plants locations come in two different datasets with no direct field to concat on. 
  # This first half calculates the distance between the given location and the plants on the first dataset and creates a dataframe to hold the information             
  distance=list(power_plants.geometry.distance(location ))                                            
  plant_distance=pd.DataFrame({'power_plant':list(power_plants['description']),'distance':distance})
  # Repeats the process for the second dataframe then concats the two dataframes                                       
  distance_2=list(power_plants_2.geometry.distance(location))
  plant_distance_2=pd.DataFrame({'power_plant':list(power_plants_2['PLANT']),'distance':distance_2})
  plant_distance=pd.concat([plant_distance,plant_distance_2])
  plant_distance.drop_duplicates('power_plant',inplace=True)
  # sorts the distances and returns the requested ammount                                           
  plant_distance=plant_distance.sort_values('distance',ascending=True)

  # Repeating for the transmission substations 
  distance=list(transmission_substations.geometry.distance(location ))                                            
  transmission_substations_distance=pd.DataFrame({'transmission_substation':list(transmission_substations['FID']),'osm_id':list(transmission_substations['osm_id']),'distance':distance})                                             

  return(plant_distance.head(n_closest),transmission_substations_distance.head(n_closest))

distance_calculator(7.5000,5.20000,5)

(                       power_plant       distance
 53                           LAGOS  813084.833260
 56                   LAGOS AIRPORT  813084.833260
 40                           IJORA  813084.833260
 1   AES Nigeria BARGE IPP Limited   835108.107576
 2     Egbin Thermal Power Station   835504.864848,
                 transmission_substation     osm_id      distance
 0  all_transmission_substations.3114602  564813837  1.426867e+06
 1  all_transmission_substations.3114625  564466620  1.431801e+06
 2  all_transmission_substations.3117563  564468439  1.449122e+06
 3  all_transmission_substations.3128584  564469422  1.455966e+06
 4  all_transmission_substations.3130639  574213499  1.473607e+06)